In [2]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O

In [5]:
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
from fastnumbers import isfloat 
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 
import bs4

import matplotlib.pyplot as plt
import seaborn as sns
import json
from tidylib import tidy_document # for tidying incorrect html

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:


def ffloat(string):
    if string is None:
        return np.nan
    if type(string)==float or type(string)==np.float64:
        return string
    if type(string)==int or type(string)==np.int64:
        return string
    return fast_float(string.split(" ")[0].replace(',','').replace('%',''),
                      default=np.nan)

def ffloat_list(string_list):
    return list(map(ffloat,string_list))

def remove_multiple_spaces(string):
    if type(string)==str:
        return ' '.join(string.split())
    return string



In [7]:
response = requests.get("http://www.example.com/", timeout=240)
response.status_code
response.content

200

b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 50px;\n        background-color: #fff;\n        border-radius: 1em;\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        body {\n            background-color: #fff;\n        }\n        div {\n            width: auto;\n            margin: 0 auto;\n            border-radius: 0;\n            padding: 1em;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\

In [8]:


url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url, timeout=240)
response.status_code
response.json()

content = response.json()
content.keys()



200

{'userId': 1,
 'id': 1,
 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}

dict_keys(['userId', 'id', 'title', 'body'])

In [9]:
def request_with_check(url):
    page_response = requests.get(url, timeout=240)
    status = page_response.status_code
    if status>299:
        raise AssertionError("page content not found, status: %s"%status)
    return page_response

In [10]:
request_with_check("https://www.google.co.in/mycustom404page")

AssertionError: page content not found, status: 404

In [11]:
request_with_check("https://www.google.co.in/")

<Response [200]>

In [12]:
from IPython.core.display import HTML
HTML("<b>Rendered HTML</b>")

In [22]:
page_response = requests.get("https://www.moneycontrol.com/india/stockpricequote/auto-2-3-wheelers/heromotocorp/HHM", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))

#HTML(str(page_content.find("div",attrs={'id':"content_full"})))

In [74]:
page_response = requests.get("https://www.screener.in/company/INFY/", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))
#print(page_content)

HTML(str(page_content.find("p",attrs={'class':"sub bigger"})))
#print(str(page_content.find("p",attrs={'class':"sub bigger"})))
HTML(str(page_content.find("li",attrs={'class':"four columns"})))
#print(str(page_content.find("li",attrs={'class':"four columns"})))
market_cap_info = page_content.find("li",attrs={'class':"four columns"})
#print(str(market_cap_info.find("b")))

#targetFile = open('page_content.txt', 'w')
#targetFile.write(str(page_content))

ratios_section = page_content.find("section",attrs={'id':"ratios"})
print(ratios_section.h2)

<h2>
  Ratios
  <small class="sub">
    
    Standalone Figures in Rs. Crores
    /
    <a class="strong" href="/company/INFY/consolidated/#ratios">
      View Consolidated
    </a>
</small>
</h2>


In [68]:
#print(page_content)
def findFullDescription_Screener(page_content):
    tag = page_content.find("p",attrs={'class':"sub bigger"})
    return tag.string

def findMarketCap_Screener(page_content):
    tag_str = str(page_content.find("li",attrs={'class':"four columns"}))
    if "Market Cap" in tag_str:
        marcap = BeautifulSoup(tag_str)
        return (str(page_content.find("b").string) + " Cr.")
    else:
        return "O Cr."



In [85]:
def get_table_simple(table,is_table_tag=True):
    elems = table.find_all('tr') if is_table_tag else get_children(table)
    table_data = list()
    for row in elems:
        
        row_data = list()
        row_elems = get_children(row)
        for elem in row_elems:
            text = elem.text.strip().replace("\n","").replace(",","")
            text = remove_multiple_spaces(text)
            if len(text)==0:
                continue
            row_data.append(text)
        table_data.append(row_data)
    return table_data
    
def get_children(html_content):
    children = list()
    for item in html_content.children:
        if type(item)==bs4.element.Comment:
            continue
        if type(item)==bs4.element.Tag or len(str(item).replace("\n","").strip())>0:
            children.append(item)
        
    return children

#print(findFullName_Screener(page_content))
#print(findMarketCap_Screener(page_content))
#print(page_content)
ratios_section = page_content.find("section",attrs={'id':"ratios"})
ratios_table = page_content.find("table",attrs={'class':"data-table"})
get_table_simple(ratios_table, is_table_tag=True)

[['Jun 2016',
  'Sep 2016',
  'Dec 2016',
  'Mar 2017',
  'Jun 2017',
  'Sep 2017',
  'Dec 2017',
  'Mar 2018',
  'Jun 2018',
  'Sep 2018',
  'Dec 2018',
  'Mar 2019'],
 ['Sales',
  '14420',
  '15000',
  '14949',
  '14920',
  '14971',
  '15356',
  '15631',
  '15984',
  '17056',
  '18297',
  '18819',
  '18935'],
 ['Expenses',
  '10402',
  '10613',
  '10532',
  '10534',
  '10635',
  '10978',
  '11166',
  '11867',
  '12616',
  '13398',
  '14227',
  '14192'],
 ['Operating Profit',
  '4018',
  '4387',
  '4417',
  '4386',
  '4336',
  '4378',
  '4465',
  '4117',
  '4440',
  '4899',
  '4592',
  '4743'],
 ['OPM %',
  '28%',
  '29%',
  '30%',
  '29%',
  '29%',
  '29%',
  '29%',
  '26%',
  '26%',
  '27%',
  '24%',
  '25%'],
 ['Other Income',
  '761',
  '763',
  '805',
  '733',
  '723',
  '849',
  '1811',
  '636',
  '716',
  '742',
  '756',
  '639'],
 ['Interest', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['Depreciation',
  '319',
  '338',
  '339',
  '336',
  '343',
  '347',
  

In [86]:
html = '''
<table>
    <tr>
        <td>Month</td>
        <td>Price</td>
    </tr>
    <tr>
        <td>July</td>
        <td>2</td>
    </tr>
    <tr>
        <td>August</td>
        <td>4</td>
    </tr>
    <tr>
        <td>September</td>
        <td>3</td>
    </tr>
    <tr>
        <td>October</td>
        <td>2</td>
    </tr>
</table>
'''
HTML(html)

Month,Price
July,2
August,4
September,3
October,2


In [87]:
pAndL_section = page_content.find("section",attrs={'id':"profit-loss"})
pAndL_table = page_content.find("table",attrs={'class':"data-table"})
get_table_simple(pAndL_table, is_table_tag=True)

[['Jun 2016',
  'Sep 2016',
  'Dec 2016',
  'Mar 2017',
  'Jun 2017',
  'Sep 2017',
  'Dec 2017',
  'Mar 2018',
  'Jun 2018',
  'Sep 2018',
  'Dec 2018',
  'Mar 2019'],
 ['Sales',
  '14420',
  '15000',
  '14949',
  '14920',
  '14971',
  '15356',
  '15631',
  '15984',
  '17056',
  '18297',
  '18819',
  '18935'],
 ['Expenses',
  '10402',
  '10613',
  '10532',
  '10534',
  '10635',
  '10978',
  '11166',
  '11867',
  '12616',
  '13398',
  '14227',
  '14192'],
 ['Operating Profit',
  '4018',
  '4387',
  '4417',
  '4386',
  '4336',
  '4378',
  '4465',
  '4117',
  '4440',
  '4899',
  '4592',
  '4743'],
 ['OPM %',
  '28%',
  '29%',
  '30%',
  '29%',
  '29%',
  '29%',
  '29%',
  '26%',
  '26%',
  '27%',
  '24%',
  '25%'],
 ['Other Income',
  '761',
  '763',
  '805',
  '733',
  '723',
  '849',
  '1811',
  '636',
  '716',
  '742',
  '756',
  '639'],
 ['Interest', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['Depreciation',
  '319',
  '338',
  '339',
  '336',
  '343',
  '347',
  